In [ ]:
# Update Bioconductor
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.17")

In [ ]:
# Install packages

# ComplexHeatmap
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("ComplexHeatmap")

In [4]:
# Load packages

library(ComplexHeatmap)